### Read in data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("C:\\Users\\Ronald\\Documents\\EECS_738\\Projects\\Says One Neuron to Another\\mushrooms.csv")
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


### Transform categorical data

In [2]:
from sklearn.preprocessing import LabelEncoder
new_df = pd.DataFrame()

le = LabelEncoder()
new_df['class'] = le.fit_transform(df['class'])
new_df['cap-shape'] = le.fit_transform(df['cap-shape'])
new_df['cap-surface'] = le.fit_transform(df['cap-surface'])
new_df['cap-color'] = le.fit_transform(df['cap-color'])
new_df['bruises'] = le.fit_transform(df['bruises'])
new_df['odor'] = le.fit_transform(df['odor'])
new_df['gill-attachment'] = le.fit_transform(df['gill-attachment'])
new_df['gill-spacing'] = le.fit_transform(df['gill-spacing'])
new_df['gill-size'] = le.fit_transform(df['gill-size'])
new_df['gill-color'] = le.fit_transform(df['gill-color'])
new_df['stalk-shape'] = le.fit_transform(df['stalk-shape'])
new_df['stalk-root'] = le.fit_transform(df['stalk-root'])
new_df['stalk-surface-above-ring'] = le.fit_transform(df['stalk-surface-above-ring'])
new_df['stalk-surface-below-ring'] = le.fit_transform(df['stalk-surface-below-ring'])
new_df['stalk-color-above-ring'] = le.fit_transform(df['stalk-color-above-ring'])
new_df['stalk-color-below-ring'] = le.fit_transform(df['stalk-color-below-ring'])
new_df['veil-type'] = le.fit_transform(df['veil-type'])
new_df['veil-color'] = le.fit_transform(df['veil-color'])
new_df['ring-number'] = le.fit_transform(df['ring-number'])
new_df['ring-type'] = le.fit_transform(df['ring-type'])
new_df['spore-print-color'] = le.fit_transform(df['spore-print-color'])
new_df['population'] = le.fit_transform(df['population'])
new_df['habitat'] = le.fit_transform(df['habitat'])

new_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


### Create activation functions for network

In [3]:
def sigmoid(x):  
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):  
    return sigmoid(x)*(1 - sigmoid(x))

def relu(x):
    return np.maximum(0,x)

def relu_derivative(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

### Define backpropagation for network

In [4]:
def back_prop(inputs, bh, bo, alpha, zh, ah, diff, zo, hw, ow):
    
    # output layer weights cost function
    dcost_dao = diff
    dao_dzo = sigmoid_derivative(zo) 
    dzo_dwo = ah
    dcost_wo = np.dot(dzo_dwo.T, dcost_dao * dao_dzo)
    
    # hidden layer weights cost function
    dcost_dzo = dcost_dao * dao_dzo
    dzo_dah = ow
    dcost_dah = np.dot(dcost_dzo , dzo_dah.T)
    dah_dzh = relu_derivative(zh) 
    dzh_dwh = inputs
    dcost_wh = np.dot(dzh_dwh.T, dah_dzh * dcost_dah)
    
    # update weights
    hw -= alpha * dcost_wh
    ow -= alpha * dcost_wo
    
    # update biases
    for d in dcost_wh:
        bh -= alpha * d
    
    for d in dcost_wo:
        bo -= alpha * d

### Shape training and testing inputs for network

In [5]:
# inputs and outputs
inputs = new_df.drop(columns=['class'])
outputs = new_df['class'].values

# 80-20 split for training and testing sets
split = round(.8 * len(inputs))

train_inputs = inputs[:split]
test_inputs = inputs[split:]
train_inputs = np.array(train_inputs)
test_inputs = np.array(test_inputs)

train_outputs = outputs[:split]
test_outputs = outputs[split:]

train_outputs = train_outputs.reshape(len(train_outputs),1)
test_outputs = test_outputs.reshape(len(test_outputs),1)

### Model training

In [6]:
# hyperparameters
np.random.seed(42)

hw = np.random.rand(len(train_inputs[0]),20)
ow = np.random.rand(20, 1)  

hw_final = 0
ow_final = 0

# bias
bh = np.random.rand(20, 1)
bh = np.transpose(bh)

bo = np.random.rand(1, 1)
bo = np.transpose(bo)

bh_final = 0
bo_final = 0

# learning rate
alpha = 0.05

for epoch in range(20):
    # hidden layer
    zh = np.dot(train_inputs, hw) + bh
    ah = relu(zh)
    
    # output layer
    zo = np.dot(ah, ow) + bo
    ao = sigmoid(zo)
    
    diff = ao - train_outputs
#     mse = (np.square(diff)).mean(axis=0)
#     print("Mean squared error is: %.2f" % mse[0])
    
    back_prop(train_inputs, bh, bo, alpha, zh, ah, diff, zo, hw, ow)
    
    # save the final weights and biases of the model for testing
    if epoch == 19:
        hw_final = hw
        ow_final = ow
        bh_final = bh
        bo_final = bo
        


### Test model accuracy

In [7]:
# hidden layer
zh = np.dot(test_inputs, hw_final) + bh_final
ah = relu(zh)
    
# output layer
zo = np.dot(ah, ow_final) + bo_final
ao = sigmoid(zo)

# test model accuracy

tp, tn, fp, fn, np, pp, pos, neg = 0, 0, 0, 0, 0, 0, 0, 0

for i in range(len(test_outputs)):
    if test_outputs[i] == 1:
        pos += 1
    else:
        neg += 1

correct = 0
for i in range(len(test_outputs)):
    prediction = int(round(ao[i][0]))
    if prediction == 1:
        pp += 1
    else:
        np += 1
    actual = test_outputs[i][0]
    if prediction == actual:
        if actual == 1:
            tp += 1
        else:
            tn += 1
    else:
        if actual == 1:
            fn += 1
        else:
            fp += 1
        
total = len(test_outputs)
accuracy = (tp + tn) / total
precision = tp / pp
true_pos_rate = tp / pos
true_neg_rate = tn / neg

print("Accuracy is: %.2f" % accuracy)
print("Precision is: %.2f" % precision)
print("True Positive Rate: %.2f" % true_pos_rate)
print("True Negative Rate: %.2f" % true_neg_rate)

Accuracy is: 0.69
Precision is: 0.69
True Positive Rate: 1.00
True Negative Rate: 0.00
